<a href="https://colab.research.google.com/github/shruti-sriniv/Prompt-Chaining/blob/main/prompt_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [1]:
!pip -q install --upgrade openai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 15.0 MB/s eta 0:00:00


In [10]:
from google.colab import userdata

In [15]:
import os
from typing import Optional, Dict

GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

print("Gemini key set? ", bool(GEMINI_API_KEY))

Gemini key set?  True


Creating a function to call the Model and provide a response

In [38]:
from dataclasses import dataclass
from typing import Optional, Dict
import os

@dataclass
class ModelProvider:
    name: str
    model: str

# Choose your default model here:
DEFAULT =  ModelProvider("gemini", "gemini-1.5-flash")

def call_model(prompt: str, provider: Optional[ModelProvider] = None, extra: Optional[Dict]=None) -> str:
    provider = provider or DEFAULT
    extra = extra or {}

    if provider.name.lower() == "gemini":
        if not GEMINI_API_KEY:
            raise RuntimeError("Set GEMINI_API_KEY in your environment (.env or Colab secret).")
        try:
            import google.generativeai as genai
        except Exception as e:
            raise RuntimeError("Install Gemini client: `pip install google-generativeai`") from e

        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel(provider.model)
        resp = model.generate_content(prompt)
        return resp.text

    else:
        raise ValueError(f"Unknown provider: {provider.name}")


Providing Input

In [39]:
article = """Researchers at Karolinska Institutet have developed nanorobots that kill cancer cells in mice. The robot’s weapon is hidden in a nanostructure and is exposed only in the tumour microenvironment, sparing healthy cells. The study is published in the journal Nature Nanotechnology. The research group at Karolinska Institutet has previously developed structures that can organise so-called death receptors on the surface of cells, leading to cell death. The structures exhibit six peptides (amino acid chains) assembled in a hexagonal pattern.
“This hexagonal nanopattern of peptides becomes a lethal weapon,” explains Professor Björn Högberg at the Department of Medical Biochemistry and Biophysics, Karolinska Institutet, who led the study. “If you were to administer it as a drug, it would indiscriminately start killing cells in the body, which would not be good. To get around this problem, we have hidden the weapon inside a nanostructure built from DNA.”
Created a ‘kill switch’
The art of building nanoscale structures using DNA as a building material is called DNA origami and is something Björn Högberg’s research team has been working on for many years. Now they have used the technique to create a ‘kill switch’ that is activated under the right conditions.

“We have managed to hide the weapon in such a way that it can only be exposed in the environment found in and around a solid tumour,” he says. “This means that we have created a type of nanorobot that can specifically target and kill cancer cells.”

The key is the low pH, or acidic microenvironment that usually surrounds cancer cells, which activates the nanorobot’s weapon. In cell analyses in test tubes, the researchers were able to show that the peptide weapon is hidden inside the nanostructure at a normal pH of 7.4, but that it has a drastic cell-killing effect when the pH drops to 6.5.

Reduced tumour growth
They then tested injecting the nanorobot into mice with breast cancer tumours. This resulted in a 70 per cent reduction in tumour growth compared to mice given an inactive version of the nanorobot.

“We now need to investigate whether this works in more advanced cancer models that more closely resemble the real human disease,” says the study's first author Yang Wang, a researcher at the Department of Medical Biochemistry and Biophysics, Karolinska Institutet. “We also need to find out what side effects the method has before it can be tested on humans.”

The researchers also plan to investigate whether it is possible to make the nanorobot more targeted by placing proteins or peptides on its surface that specifically bind to certain types of cancer.

The research was funded by the Knut and Alice Wallenberg Foundation, the European Research Council (ERC), the Swedish Research Council and the Academy of Finland. The invention will be patented"""

print(article)

Researchers at Karolinska Institutet have developed nanorobots that kill cancer cells in mice. The robot’s weapon is hidden in a nanostructure and is exposed only in the tumour microenvironment, sparing healthy cells. The study is published in the journal Nature Nanotechnology. The research group at Karolinska Institutet has previously developed structures that can organise so-called death receptors on the surface of cells, leading to cell death. The structures exhibit six peptides (amino acid chains) assembled in a hexagonal pattern.
“This hexagonal nanopattern of peptides becomes a lethal weapon,” explains Professor Björn Högberg at the Department of Medical Biochemistry and Biophysics, Karolinska Institutet, who led the study. “If you were to administer it as a drug, it would indiscriminately start killing cells in the body, which would not be good. To get around this problem, we have hidden the weapon inside a nanostructure built from DNA.”
Created a ‘kill switch’
The art of buildi

In [40]:
# Let us start by creating a summary of some data

summary_prompt = f"""You are a concise summarizer. Summarize the following article in 5-10 bullet points

ARTCLE: {article}
"""

summary = call_model(summary_prompt)
print("Summary:\n", summary)

Summary:
 * Researchers at Karolinska Institutet created DNA origami nanorobots that target and kill cancer cells.
* The nanorobots contain a "kill switch" – peptides that induce cell death.
* The peptides are inactive at normal pH (7.4) but activated in the acidic environment (pH 6.5) of tumors.
* In mice with breast cancer, the nanorobots reduced tumor growth by 70%.
* The nanorobots' targeting mechanism relies on the acidic tumor microenvironment.
*  Further research will focus on testing in more advanced cancer models and assessing side effects.
*  Future development includes enhancing targeting specificity by adding surface proteins/peptides.
* The research was funded by several organizations and the invention will be patented.
* The study is published in Nature Nanotechnology.




Generating a chained prompt leveraging prompt 1

In [46]:
quiz_prompt = f""" Using the summary below, generate ten quiz questions with exactly four options (A, B, C, D) and mark the right answer

SUMMARY:
{summary}
"""

quiz = call_model(quiz_prompt)

print("POP QUIZ\n", quiz)

POP QUIZ
 Here are ten multiple-choice quiz questions based on the provided summary, each with four options (A, B, C, D) and the correct answer marked with an asterisk (*):

1. What type of technology did the researchers at Karolinska Institutet develop to target cancer cells?
    A)  Genetically modified viruses
    B)  DNA origami nanorobots *
    C)  Targeted chemotherapy drugs
    D)  Immunotherapy agents

2. What triggers the activation of the "kill switch" within the nanorobots?
    A)  Exposure to sunlight
    B)  Interaction with healthy cells
    C)  The acidic environment of tumors *
    D)  High concentrations of oxygen

3.  What is the approximate pH of the tumor microenvironment that activates the nanorobots' "kill switch"?
    A)  7.4
    B)  8.0
    C)  6.5 *
    D)  5.0

4.  In mice with breast cancer, what was the approximate percentage reduction in tumor growth observed after treatment with the nanorobots?
    A)  30%
    B)  50%
    C)  70% *
    D)  90%

5. The nano

Generating another chained prompt based on prompt 2

In [47]:
study_guide_prompt = f""" Using the quiz below, create a comprehensive study guide with appropriate headers and bullet points for a student attempting to take this quiz.

QUIZ:
{quiz}
"""

study_guide = call_model(study_guide_prompt)

print("STUDY GUIDE\n", study_guide)

STUDY GUIDE
 ## Study Guide: DNA Origami Nanorobots for Cancer Treatment

This study guide covers the key concepts from the provided quiz on DNA origami nanorobots used in cancer treatment.  Use it to prepare for your quiz.


**I. Nanorobot Technology:**

* **Type of Technology:** The researchers utilized **DNA origami nanorobots** to target cancer cells.  This is a distinct approach from genetically modified viruses, targeted chemotherapy, or immunotherapy agents.

* **Targeting Mechanism:** The nanorobots primarily rely on the **acidic tumor microenvironment** for targeting.  They don't primarily use antibodies, magnetic fields, or radioactive isotopes.

* **"Kill Switch" Activation:** The nanorobots' "kill switch" is activated by the **acidic pH** (approximately 6.5) of the tumor microenvironment.  This differs from activation by sunlight, interaction with healthy cells, or high oxygen concentrations.

* **Cell Death Induction:** The **peptides** within the nanorobot are responsible

Creating a function for prompt chaining

In [55]:
# The function uses an optional variable called context. This can be the training data for a prmpt (such as the artcile for the summary prompt), or it can be the result of another prompt
def prompt_chain(text, context : Optional[str]):
  if context:
        formatted_text = f""" {text}
    CONTEXT: {context}
    """
  else:
        formatted_text = text

  result = call_model(formatted_text)
  print("RESULT\n", result)

In [56]:
# Calling the function on the summary prompt
prompt_chain(summary_prompt, article)

RESULT
 * Karolinska Institutet researchers developed DNA origami-based nanorobots that target and kill cancer cells in mice.
* The nanorobots contain a "lethal weapon" – a hexagonal pattern of peptides – hidden within a DNA nanostructure.
* This weapon is activated only in the acidic environment (low pH) surrounding tumors, sparing healthy cells.
*  In lab tests, the nanorobots showed a drastic cell-killing effect at pH 6.5, but were inactive at normal pH 7.4.
*  Mouse studies showed a 70% reduction in breast cancer tumor growth.
*  Further research is needed to test the nanorobots in more advanced cancer models and assess potential side effects before human trials.
*  Future plans include improving the nanorobot's targeting specificity by adding proteins or peptides.
* The research was funded by several organizations, and the invention will be patented.




In [58]:
# Calling the function using an exisiting prompt's result as context
prompt_chain(quiz_prompt, summary)

RESULT
 Here are ten quiz questions based on the provided summary, each with four options (A, B, C, D) and the correct answer marked with an asterisk (*):

1.  Where were the DNA origami nanorobots developed?
    A) MIT  B) Stanford University  C) Oxford University  *D) Karolinska Institutet*

2.  What is the primary function of the peptides within the nanorobots?
    A) To enhance DNA replication  B) To improve targeting accuracy  *C) To induce cell death*  D) To neutralize tumor acidity

3.  At what pH level are the peptides within the nanorobots activated?
    A) 7.4  *B) 6.5*  C) 8.0  D) 5.0

4.  In the mouse model study, what percentage reduction in tumor growth was observed?
    A) 30%  B) 50%  *C) 70%*  D) 90%

5.  What is the main targeting mechanism of the nanorobots?
    A) Specific antibodies  B) Magnetic fields  *C) Acidic tumor microenvironment*  D) Radio waves

6.  What is the next stage of research, as mentioned in the summary?
    A) Human clinical trials  B) Testing in